In [4]:
import os
import re
import numpy as np
import pandas as pd
import sys
from pathlib import Path
from datetime import date, datetime,timedelta
from calendar import monthrange
from IPython.display import display,HTML
pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", False)

DATA_PROCESSING_DIR = Path("/Users/ylin/ValtaWork/Data and Reporting")
sys.path.append(str(DATA_PROCESSING_DIR))

from DataProcessing import *
data = import_data()
property=property_input()

In [11]:
startdate = pd.to_datetime('2025-12-01')
enddate = pd.to_datetime('2026-01-31')
osbr = data[data["Listing"].str.contains("Cottage")]
mask = (osbr["checkin_date"]>=startdate) & (osbr["checkin_date"]<=enddate) |\
(osbr["checkout_date"]>=startdate) & (osbr["checkout_date"]<=enddate)
osbr = osbr.loc[mask]
checkins = osbr.assign(Date = osbr["checkin_date"].astype(str).str[:10]).groupby("Date", as_index=False).agg(checkin=("checkin_date",'count'))
checkouts = osbr.assign(Date = osbr["checkout_date"].astype(str).str[:10]).groupby("Date", as_index=False).agg(checkout=("checkout_date",'count'))
counts = checkins.merge(checkouts,on="Date",how="outer")
df = pd.DataFrame({
    "Date": pd.date_range(startdate, enddate)
})
counts["Date"] = pd.to_datetime(counts["Date"])
counts = df.merge(counts,on="Date",how="left")
counts["Weekday"] = counts["Date"].dt.day_name()
counts["Date"] = counts["Date"].astype(str).str[:10]
counts = counts[["Date", "Weekday", "checkin", "checkout"]]

In [12]:
property.loc[property["Listing"]=="Cottage 12",'BEDROOMS'] = 3
osbr_sum = (osbr[["Listing","checkout_date"]].merge(property[["Listing","BEDROOMS"]],on="Listing",how="left")
            .pivot_table(index="checkout_date", columns="BEDROOMS",aggfunc="size",fill_value=0).reset_index()
            .assign(Date=lambda x: x["checkout_date"].astype(str).str[:10]))

counts = counts.merge(osbr_sum, on="Date", how="left").drop(columns=["checkout_date"])
out ={
    "counts": counts,
    "bookings": osbr
}
with pd.ExcelWriter("/Users/ylin/My Drive/Cohost/Cohost Cleaner Compensation/Working/Brittany_booking_counts.xlsx", engine="openpyxl") as ew:
     for sheet_name, df in out.items():
        sheet = sheet_name[:31]
        df.to_excel(ew, sheet_name=sheet, index=False, na_rep="")

#counts.to_excel("/Users/ylin/My Drive/Cohost/Cohost Cleaner Compensation/Working/Brittany_booking_counts.xlsx", index=False,na_rep="")

In [8]:
osbr

,Listing,Confirmation.Code,guests,adults,children,infants,checkin_date,checkout_date,nights,earnings,...,accommodation_fare,booking_platform,lead_time,confirmation_date,DailyListingPrice,AvgDailyRate,total_revenue,yearmonth,checkin_date_plot,Term
10539,Cottage 3,GY-8TQjxyPX,2.0,2.0,0.0,0.0,2025-11-26,2025-12-01,5.0,1240.09,...,945.00,Manual,65.0,2025-09-22,248.018000,189.000000,1240.09,2025-11,2025-11-26,STR
10544,Cottage 2,HMYXPMSEJB,2.0,2.0,0.0,0.0,2025-11-30,2025-12-02,2.0,198.57,...,190.00,Airbnb,19.0,2025-11-11,99.285000,95.000000,198.57,2025-11,2025-11-30,STR
10546,Cottage 2,BC-4qvMxByD1,2.0,2.0,0.0,0.0,2025-12-02,2025-12-03,1.0,143.20,...,93.22,Booking.com,4.0,2025-11-28,143.200000,93.220000,143.20,2025-12,2025-12-02,STR
10550,Cottage 10,BC-3vmPOBBZr,2.0,2.0,0.0,0.0,2025-12-02,2025-12-04,2.0,250.27,...,186.44,Booking.com,2.0,2025-11-30,125.135000,93.220000,250.27,2025-12,2025-12-02,STR
10553,Cottage 5,BC-B7nRm9lEx,3.0,3.0,0.0,0.0,2025-12-02,2025-12-04,2.0,354.62,...,277.30,Booking.com,20.0,2025-11-12,177.310000,138.650000,354.62,2025-12,2025-12-02,STR
10556,Cottage 3,BC-lkrq008Wg,2.0,2.0,0.0,0.0,2025-12-03,2025-12-05,2.0,252.33,...,186.44,Booking.com,5.0,2025-11-28,126.165000,93.220000,252.33,2025-12,2025-12-03,STR
10558,Cottage 4,HM39SKBWA5,1.0,1.0,0.0,0.0,2025-12-02,2025-12-05,3.0,354.06,...,354.00,Airbnb,12.0,2025-11-20,118.020000,118.000000,354.06,2025-12,2025-12-02,STR
10562,Cottage 6,HM3XQB9TCM,2.0,2.0,0.0,0.0,2025-12-02,2025-12-06,4.0,478.21,...,428.00,Airbnb,51.0,2025-10-12,119.552500,107.000000,478.21,2025-12,2025-12-02,STR
10564,Cottage 12,GY-vZisHydN,4.0,4.0,0.0,0.0,2025-12-04,2025-12-06,2.0,458.26,...,334.00,Manual,86.0,2025-09-09,229.130000,167.000000,458.26,2025-12,2025-12-04,STR
10565,Cottage 5,GY-3MLkkDsL,3.0,3.0,1.0,1.0,2025-12-04,2025-12-07,3.0,576.70,...,437.00,Manual,21.0,2025-11-13,192.233333,145.666667,576.70,2025-12,2025-12-04,STR


In [34]:
ratings_path = "/Users/ylin/My Drive/Cohost/** Properties ** -- Valta/0_Cohosting/1-Reviews/Guesty reviews from Tech team/20251201 guesty_reviews.xlsx"
ratings = pd.read_excel(ratings_path)
ratings["createdAt"] = pd.to_datetime(ratings["createdAt"], errors="coerce")
idx = (
    ratings["nickname"].str.contains("Cottage", na=False)
    & (ratings["createdAt"] > pd.to_datetime("2025-10-31"))
    & (ratings["createdAt"] < pd.to_datetime("2025-12-01")) 
    & (ratings["Overall"].isin([5,10]))
)
ratings.loc[idx]

,Unnamed: 0,reservationId,channelId,createdAt,nickname,Property,Guest name,Check in,Check out,Accuracy,...,Location,Value,Overall,Public Review,Reservation,Booking.com Rating,Booking.com Positive content,Booking.com Negative content,Cohost review,Cohost
7,7,691563075fbb68004458b541,airbnb2,2025-11-25,Cottage 2,NaN,Sean,2025-11-14,2025-11-15,5.0,...,5.0,5.0,5.0,Cabin was just like pictures. Very well kept. Enjoyed the rec room which was really nice space. Loved the Grayland area and was a quiet neighborhood. I look forward to another visit there next year.,HMNCCWNWXQ,NaN,NaN,NaN,0,NaN
10,10,68edc8ec2bcd3a001a0c8af8,bookingCom,2025-11-23,Cottage 2,NaN,Lisa,2025-11-19,2025-11-21,10.0,...,10.0,10.0,10.0,NaN,BC-kg2ngMLPr,10.0,NaN,NaN,0,NaN
14,14,690c5f5b4c2c810011a96d1d,airbnb2,2025-11-21,Cottage 2,NaN,Brayden,2025-11-15,2025-11-17,4.0,...,5.0,4.0,5.0,"It was definitely worth the cost and the views are amazing, beach access close by as well so that was a plus!! Overall we really enjoyed our stay!",HMTCE9DDPZ,NaN,NaN,NaN,0,NaN
18,18,69094bd0f27bbb00131bdef3,airbnb2,2025-11-18,Cottage 6,NaN,Ashley,2025-11-08,2025-11-09,5.0,...,5.0,5.0,5.0,Perfect stay for our weekend crabbing trip! Cozy and clean!,HMSAT2A3DH,NaN,NaN,NaN,0,NaN
40,40,68d7697ac3466000126697e4,airbnb2,2025-11-12,Cottage 1,NaN,Becky,2025-11-08,2025-11-11,5.0,...,5.0,5.0,5.0,What a wonderful get away! The cottage was perfectly cozy and I love having the beach so close!,HMZFATXTK3,NaN,NaN,NaN,0,NaN
49,49,6907a8c397f5d2000f218008,airbnb2,2025-11-11,Cottage 4,NaN,Lori,2025-11-07,2025-11-08,5.0,...,5.0,5.0,5.0,"Comfortable beds, nice towels, cute place would stay there again.",HMWT8WZX55,NaN,NaN,NaN,0,NaN
51,51,690e6ad669fb630013910e70,airbnb2,2025-11-11,Cottage 11 (tiny),NaN,Chase,2025-11-08,2025-11-10,5.0,...,5.0,5.0,5.0,Beautiful little unit for the weekend! loved the soaker tup! Thanks for another great stay!,HMBPAF3TYH,NaN,NaN,NaN,0,NaN
55,55,690a9c4c6ba07f0012486e1c,homeaway2-Vrbo,2025-11-09,Cottage 12,NaN,James,2025-11-07,2025-11-09,5.0,...,5.0,NaN,5.0,Host was very helpful. We loved the close access to the beach. Pet friendly is always helpful! We enjoyed our stay!,HA-IHBK1XY,NaN,NaN,NaN,0,NaN
59,59,68c9a382ea829a00322ff3e8,airbnb2,2025-11-08,Cottage 2,NaN,Jeff,2025-11-04,2025-11-07,5.0,...,5.0,5.0,5.0,With king tides and great razor clamming I couldn't ask for more.,HMYYEQK9J2,NaN,NaN,NaN,0,NaN
61,61,68ee6bfeec7cd2001241df75,airbnb2,2025-11-07,Cottage 6,NaN,Tim,2025-11-04,2025-11-07,5.0,...,5.0,5.0,5.0,"It is alway great to stay at Ocean Spray. We have stayed in many different cabins, but do have our favorite. They are all good.\nThey are always one step ahead of us on all communications\nAgain, an awesome stay\nThank You",HMAZ33APJH,NaN,NaN,NaN,0,NaN
